In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Pic2Word

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_pic2word.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_pic2word.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_pic2word.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

This notebook demonstrates how to use the [Pic2Word](https://github.com/google-research/composed_image_retrieval) model in Vertex AI Model Garden.

### Objective

Following the notebook you will conduct experiments using the pre-built docker image on Vertex AI.

- Deploy pretrained Pic2Word models in Google Cloud Vertex AI

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Model Registry
- Vertex AI Online Prediction

### Dataset

We use the COCO validation set (5,000 images) for evaluation.

### Costs

This tutorial uses billable components of Google Cloud:

- Vertex AI
- Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing?_ga=2.46650789.-341051769.1686949237) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing?_ga=2.46650789.-341051769.1686949237), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/?_ga=2.247379078.-341051769.1686949237) to generate a cost estimate based on your projected usage.

## Before you begin

### Colab only
Run the following commands for Colab and skip this section if you are using Workbench or Colab Enterprise.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs.

1. [Create a service account](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) with **Vertex AI User** and **Storage Object Admin** roles for deploying the model to a Vertex AI endpoint.[link text](https://)

### Import the necessary packages

In [ ]:
import os
import sys

from google.cloud import aiplatform

Set the following variables for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the specified region (`REGION`). Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1").

In [ ]:
# Cloud project ID.
PROJECT_ID = ""  # @param {type:"string"}

# Region for launching jobs.
# TPU deployment is only supported in us-west1.
REGION = ""  # @param {type:"string"}

# Cloud Storage bucket for storing experiment outputs.
# Start with gs:// prefix, e.g. gs://foo_bucket.
BUCKET_URI = "gs://"  # @param {type:"string"}
assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(BUCKET_URI, "gemma")

# Initialize Vertex AI API.
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# The service account looks like:
# '@.iam.gserviceaccount.com'
# Please visit https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create a service account with `Vertex AI User` and `Storage Object Admin` roles.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

# Provision GCS bucket permissions to the SERVICE_ACCOUNT.
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID
! gcloud services enable language.googleapis.com

### Define common variables

In [ ]:
# Prediction constants.
PREDICTION_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pic2word-serve:latest"
PREDICTION_ACCELERATOR_TYPE = "NVIDIA_TESLA_T4"
PREDICTION_MACHINE_TYPE = "n1-standard-8"

## Deploy model for online prediction

This section uploads the model to Vertex Model Registry and deploys it on an Endpoint resource. This will take around 15 minutes to finish.

### Upload and deploy model to Vertex AI

In [ ]:
# Upload model.
serving_env = {"MODEL_ID": "googleai-pic2word", "DEPLOY_SOURCE": "notebook"}

model = aiplatform.Model.upload(
    display_name="pic2word-model",
    serving_container_image_uri=PREDICTION_DOCKER_URI,
    serving_container_ports=[7080],
    serving_container_predict_route="/predictions/pic2word",
    serving_container_health_route="/ping",
    serving_container_environment_variables=serving_env,
)

# Create an endpoint.
endpoint = aiplatform.Endpoint.create(display_name="pic2word-endpoint")

# Deploy model to endpoint.
model.deploy(
    endpoint=endpoint,
    machine_type=PREDICTION_MACHINE_TYPE,
    accelerator_type=PREDICTION_ACCELERATOR_TYPE,
    accelerator_count=1,
    traffic_percentage=100,
    service_account=SERVICE_ACCOUNT,
)

You can manage your uploaded models in the [Model Registry](https://console.cloud.google.com/vertex-ai/models) and your endpoints in the [Endpoints](https://console.cloud.google.com/vertex-ai/endpoints).

## Send a prediction request to the endpoint

Set the query text and upload your desired query image to the specified INPUT_BUCKET. Use an asterisk in the query text to specify which token gets replaced by the image token. The retrieved images will be uploaded to the specified output bucket.

In [ ]:
# Text prompt.
PROMPT = "a bunch of *"

# File name of your query image. Example: "cat.png".
IMAGE_FILE_NAME = "cat.png"

# gs:// path to the directory where the query image is stored.
INPUT_BUCKET = os.path.join(BUCKET_URI, "input")

# gs:// path to the directory where the result images will be uploaded to.
OUTPUT_BUCKET = os.path.join(BUCKET_URI, "output")

instances = [
    {
        "prompts": PROMPT,
        "image_path": INPUT_BUCKET,
        "image_file_name": IMAGE_FILE_NAME,
        "output_storage_dir": OUTPUT_BUCKET,
    },
]

response = endpoint.predict(instances=instances)
print(response)

## Cleaning Up

To clean up all Google Cloud resources used in this project, you can delete the Google Cloud project you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Delete endpoint resource.
endpoint.delete(force=True)

# Delete model resource.
model.delete()

# Delete Cloud Storage objects that were created.
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI